<a href="https://colab.research.google.com/github/Sathiyan97/HTML-Practice/blob/master/Count_wheat_spikes_using_finetuned_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/FYP-Data/Wheat Spikes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/drive/MyDrive/FYP-Data/Wheat Spikes/train.csv
/content/drive/MyDrive/FYP-Data/Wheat Spikes/my_model-2.pth
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train.gsheet
/content/drive/MyDrive/FYP-Data/Wheat Spikes/my_model-3.pth
/content/drive/MyDrive/FYP-Data/Wheat Spikes/custom-data-eval.csv
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2171_1.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2172_4.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2181_4.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2171_2.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2172_3.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2163_0.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2171_4.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2163_3.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2181_3.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2171_5.jpg
/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/2152_3.jpg
/content/drive/MyDr

In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
import random
from PIL import Image,ImageDraw
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import pandas as pd
import numpy as np
import cv2
import os
import re
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torchvision

# Load Data

In [5]:
train=pd.read_csv('/content/drive/MyDrive/FYP-Data/Wheat Spikes/train.csv')

In [6]:
train.head()

,image_id,width,height,bbox,source
0,1011_0,1024,1024,"[155, 500, 113, 68]",extend_set
1,1011_0,1024,1024,"[218, 552, 125, 89]",extend_set
2,1011_0,1024,1024,"[352, 547, 95, 110]",extend_set
3,1011_0,1024,1024,"[543, 536, 48, 88]",extend_set
4,1011_0,1024,1024,"[653, 535, 68, 64]",extend_set


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70939 entries, 0 to 70938
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  70939 non-null  object
 1   width     70939 non-null  int64 
 2   height    70939 non-null  int64 
 3   bbox      70939 non-null  object
 4   source    70939 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.7+ MB


# Data preprocessing

In [8]:
coords=pd.DataFrame(list(train.bbox.apply(lambda x:x[1:-1].split(',')).values), columns=['x1','y1','w','h'])

In [9]:
coords.head()

,x1,y1,w,h
0,155,500,113,68
1,218,552,125,89
2,352,547,95,110
3,543,536,48,88
4,653,535,68,64


In [10]:
df=pd.concat([train,coords],axis=1)
df.head()

,image_id,width,height,bbox,source,x1,y1,w,h
0,1011_0,1024,1024,"[155, 500, 113, 68]",extend_set,155,500,113,68
1,1011_0,1024,1024,"[218, 552, 125, 89]",extend_set,218,552,125,89
2,1011_0,1024,1024,"[352, 547, 95, 110]",extend_set,352,547,95,110
3,1011_0,1024,1024,"[543, 536, 48, 88]",extend_set,543,536,48,88
4,1011_0,1024,1024,"[653, 535, 68, 64]",extend_set,653,535,68,64


In [11]:
df['x1']=pd.to_numeric(df['x1'])
df['y1']=pd.to_numeric(df['y1'])
df['w']=pd.to_numeric(df['w'])
df['h']=pd.to_numeric(df['h'])


In [12]:
df['x2']=df['x1']+df['w']
df['y2']=df['y1']+df['h']


In [13]:
df.drop(['bbox','width','height','w','h','source'],axis=1,inplace=True)

In [14]:
df.head()

,image_id,x1,y1,x2,y2
0,1011_0,155,500,268,568
1,1011_0,218,552,343,641
2,1011_0,352,547,447,657
3,1011_0,543,536,591,624
4,1011_0,653,535,721,599


In [15]:
unique_imgs=df.image_id.unique()

In [16]:
unique_imgs

array(['1011_0', '1011_1', '1011_2', ..., '6202_3', '6202_4', '6202_5'],
      dtype=object)

In [17]:
train_transforms = T.Compose([
    T.RandomHorizontalFlip(0.5),
    # T.RandomVerticalFlip(0.5),
    # T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor()
])

val_transforms = T.Compose([
    T.ToTensor()
])


In [18]:
# class CustData(torch.utils.data.Dataset):
#     def __init__(self,df,unique_imgs,indices):
#         self.df=df
#         self.unique_imgs=unique_imgs
#         self.indices=indices
#     def __len__(self):
#         return len(self.indices)

#     def __getitem__(self,idx):
#         image_name=self.unique_imgs[self.indices[idx]]
#         boxes=self.df[self.df.image_id==image_name].values[:,1:].astype('float')
#         img=Image.open('/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/'+image_name+'.jpg').convert('RGB')
#         labels=torch.ones((boxes.shape[0]),dtype=torch.int64)
#         target={}
#         target['boxes']=torch.tensor(boxes)
#         target['labels']=labels
#         return T.ToTensor()(img),target

class CustData(torch.utils.data.Dataset):
  def __init__(self, df, unique_imgs, indices, transforms=None):
      self.df = df
      self.unique_imgs = unique_imgs
      self.indices = indices
      self.transforms = transforms

  def __len__(self):
      return len(self.indices)

  def __getitem__(self, idx):
      image_name = self.unique_imgs[self.indices[idx]]
      boxes = self.df[self.df.image_id == image_name].values[:, 1:].astype('float')
      img=Image.open('/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/'+image_name+'.jpg').convert('RGB')

      if self.transforms:
          img = self.transforms(img)

      labels = torch.ones((boxes.shape[0]), dtype=torch.int64)
      target = {
            'boxes': torch.tensor(boxes),
            'labels': labels
        }

      return img, target


# Train test spliting

In [19]:
train_inds,val_inds=train_test_split(range(unique_imgs.shape[0]),test_size=0.1)

In [20]:
def custom_collate(data):
    return data

In [21]:
# train_dl=torch.utils.data.DataLoader( CustData(df,unique_imgs,train_inds),
#                                      batch_size=15,
#                                      shuffle=True,
#                                      collate_fn=custom_collate,
#                                      pin_memory=True if torch.cuda.is_available() else False

#                                     )
# val_dl=torch.utils.data.DataLoader( CustData(df,unique_imgs,val_inds),
#                                      batch_size=8,
#                                      shuffle=True,
#                                      collate_fn=custom_collate,
#                                      pin_memory=True if torch.cuda.is_available() else False

#                                     )
train_dl = torch.utils.data.DataLoader(
    CustData(df, unique_imgs, train_inds, transforms=train_transforms),
    batch_size=15,
    shuffle=True,
    collate_fn=custom_collate,
    pin_memory=True if torch.cuda.is_available() else False
)

val_dl = torch.utils.data.DataLoader(
    CustData(df, unique_imgs, val_inds, transforms=val_transforms),
    batch_size=8,
    shuffle=True,
    collate_fn=custom_collate,
    pin_memory=True if torch.cuda.is_available() else False
)


In [22]:

train_dl

# Model training

In [23]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)  #  FPN (Feature Pyramid Network)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:03<00:00, 47.8MB/s]


In [24]:
# model

In [25]:
num_classes =2 # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [26]:
# model

In [27]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
device

device(type='cuda')

In [29]:
optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9,weight_decay=0.0005)
num_epochs=20

In [ ]:
model.to(device)
for epoch in range(num_epochs):
    epoch_loss = 0
    for data in train_dl:
        imgs = []
        targets = []

        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ['boxes'] = d[1]['boxes'].to(device)
            targ['labels'] = d[1]['labels'].to(device)
            targets.append(targ)

        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(epoch_loss)

    if (epoch + 1) % 5 == 0 or epoch + 1 == num_epochs:
        # Save the weights at specific epochs
        torch.save(model.state_dict(), f'weights_epoch_{epoch + 1}.pt')


146.4006916645739
126.63468965257466
123.30521166079194
121.34207611605798
119.28390920838388
117.11042812297886
116.20364075964518
115.91171931233778


# model Validation

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/FYP-Data/Wheat Spikes/my_model-after-img-augmentation.pth')

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/FYP-Data/Wheat Spikes/my_model-after-img-augmentation.pth'))

In [ ]:
model.eval()
data=iter(val_dl).__next__()

In [ ]:
# model.eval()

In [ ]:
img=data[0][0]
boxes=data[0][1]['boxes']
labels=data[0][1]['labels']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
output=model([img.to(device)])

In [ ]:
output

In [ ]:
out_box=output[0]['boxes']
out_scores=output[0]['scores']

In [ ]:
out_box.shape,out_scores.shape

In [ ]:
keep=torchvision.ops.nms(out_box,out_scores,0.4)

In [ ]:
out_box.shape,keep.shape

In [ ]:
im=(img.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')

In [ ]:
# im

# Model results

In [ ]:
vsample=Image.fromarray(im)

In [ ]:
from PIL import Image, ImageDraw

# Assuming you have defined `vsample`, `keep2`, and `out_box1` variables

draw = ImageDraw.Draw(vsample)
x = 0
for idx in keep:
    x += 1
    box = out_box[idx]
    draw.rectangle(list(box), fill=None, outline='white')

    # Calculate the center point of the rectangle
    center_x = (box[0] + box[2]) // 2
    center_y = (box[1] + box[3]) // 2

    # Add a red dot at the center point
    radius = 3
    dot_box = (center_x - radius, center_y - radius, center_x + radius, center_y + radius)
    draw.ellipse(dot_box, fill='red')

vsample

In [ ]:
draw=ImageDraw.Draw(vsample)
x = 0
for i in keep:
    x +=1
    box = out_box[i]
    draw.rectangle(list(box),fill=None,outline='white')
vsample

In [ ]:
print(x)

# Testing custom image

In [ ]:
img1=Image.open('/content/drive/MyDrive/FYP-Data/Wheat Spikes/train/1051_1.jpg').convert('RGB')

In [ ]:
# Create a copy of the image to draw bounding boxes on
image_with_boxes = img1.copy()

In [ ]:
transform = T.Compose([T.ToTensor()])
input_image = transform(img1)

In [ ]:
type(input_image)

In [ ]:
output1=model([input_image.to(device)])

In [ ]:
out_box1=output1[0]['boxes']
out_scores1=output1[0]['scores']

In [ ]:
out_box1.shape,out_scores1.shape

In [ ]:
keep2=torchvision.ops.nms(out_box1,out_scores1,0.06)

In [ ]:
out_box1.shape,keep2.shape

In [ ]:
im1=(input_image.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')

In [ ]:
vsample=Image.fromarray(im1)

In [ ]:
draw=ImageDraw.Draw(vsample)
x = 0
for idx in keep2:
    x +=1
    box = out_box1[idx]
    draw.rectangle(list(box),fill=None,outline='white')
vsample

In [ ]:
print(x)

# Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/FYP-Data/Wheat Spikes/custom-data-eval.csv")

In [ ]:
actual = test_df["actual"]
predicted = test_df["predicted"]

### Mean Absolute Error (MAE) | Mean Squared Error (MSE) | Root Mean Squared Error (RMSE)



In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(actual, predicted)
mse = mean_squared_error(actual, predicted)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MSE) : ", mae)
print("Mean Squared Error (MSE) : ", mse)
print("Root Mean Squared Error (RMSE) : ", rmse)

### Predtiction Plot

In [ ]:
# Calculate the line of best fit
slope, intercept = np.polyfit(actual, predicted, 1)

# Create the scatter plot
plt.scatter(actual, predicted)

# Add the line of best fit
plt.plot(actual, slope*actual + intercept, 'r')

plt.xlabel("Actual count")
plt.ylabel("Predicted count")
plt.title("Finetuned Faster RCNN comparison")

plt.show()

### Residual plot

In [ ]:
residuals = [actual - predicted for actual, predicted in zip(actual, predicted)]
plt.scatter(predicted, residuals)
plt.xlabel('Predicted Counts')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Counts')
plt.show()


### Histogram of Residuals

In [ ]:
plt.hist(residuals, bins=20)
plt.xlabel('Residuals')
plt.title('Histogram of Residuals')
plt.show()
